In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

## INICIAL

In [3]:
RESULTS_DIR = "results/Educacion/educacion_inicial"
INPUT_FILE = "data/consolidated/consolidado_educacion.xlsx"


df = pd.read_excel(INPUT_FILE)


df["precio_por_unidad_estandar"] = pd.to_numeric(
    df["precio_por_unidad_estandar"], errors="coerce"
)

df = df.dropna(subset=["precio_por_unidad_estandar"]).copy()


productos_inicial = [
    "cartulinas",
    "colores",
    "crayones-y-oleos",
    "cuaderno triple reglon",
    "limpiatipos",
    # "loncheras",
    "plastilinas",
    "reglas",
]

df["producto_especifico"] = df["producto_especifico"].astype(str)
df = df[df["producto_especifico"].isin(productos_inicial)].copy()

# marcar que todo es INICIAL
df["nivel_educativo"] = "INICIAL"

# ===============================
if "fecha_scraping" in df.columns:
    df["FECHA"] = pd.to_datetime(df["fecha_scraping"], errors="coerce")
else:
    df["FECHA"] = pd.NaT

# ===============================
vol_prod = (
    df.groupby("producto_especifico")["precio_por_unidad_estandar"]
      .std()
      .reset_index(name="VOLATILIDAD")
)

print("Volatilidad por producto - INICIAL (precio directo):")
print(vol_prod)

# Heatmap: una fila con N productos (estilo salud)
fig_vol = go.Figure(
    data=go.Heatmap(
        z=[vol_prod["VOLATILIDAD"].values],
        x=vol_prod["producto_especifico"],
        y=["Volatilidad"],
        colorscale="Teal",
        colorbar=dict(title="Volatilidad"),
        showscale=True,
    )
)

# Anotar valores dentro de cada celda
for _, row in vol_prod.iterrows():
    fig_vol.add_trace(
        go.Scatter(
            x=[row["producto_especifico"]],
            y=["Volatilidad"],
            text=[f"{row['VOLATILIDAD']:.2f}"],
            mode="text",
            showlegend=False,
        )
    )

fig_vol.update_layout(
    title=None,
    xaxis_title="Producto (INICIAL)",
    yaxis_title="",
    template="plotly_white",
    width=900,
    height=500,
    margin=dict(l=40, r=40, t=40, b=120),
)
fig_vol.show()
fig_vol.write_image("results/Educacion/educacion_inicial/volatilidad_por_producto_inicial.pdf")


# ===============================
# 5. Boxplot de precios por producto (precio directo, estilo salud)
# ===============================
fig_box = px.box(
    df,
    x="producto_especifico",
    y="precio_por_unidad_estandar",
    color="producto_especifico",
    labels={
        "producto_especifico": "Producto (INICIAL)",
        "precio_por_unidad_estandar": "Precio por unidad estándar (S/)",
    },
    width=900,
    height=500,
    template="plotly_white",
)

fig_box.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,
    title=None,
    margin=dict(l=40, r=40, t=40, b=120),
)

# Ordenar productos por valor total (como en el ejemplo salud)
fig_box.update_layout(xaxis={"categoryorder": "total descending"})

# Rango del eje Y (opcional, tipo ejemplo salud)
max_price = df["precio_por_unidad_estandar"].max()
fig_box.update_yaxes(range=[0, max_price * 1.1])

fig_box.show()
fig_box.write_image("results/Educacion/educacion_inicial/boxplot_precio_por_producto_inicial.pdf")

# ===============================
# 6. Evolución del precio promedio por producto (precio directo)
# ===============================
if df["FECHA"].notna().any():
    evol_prod = (
        df.groupby(["FECHA", "producto_especifico"])["precio_por_unidad_estandar"]
          .mean()
          .reset_index()
    )

    fig_line = px.line(
        evol_prod,
        x="FECHA",
        y="precio_por_unidad_estandar",
        color="producto_especifico",
        markers=True,
        labels={
            "FECHA": "Fecha",
            "precio_por_unidad_estandar": "Precio promedio (S/)",
            "producto_especifico": "Producto (INICIAL)",
        },
        width=900,
        height=500,
        template="plotly_white",
    )

    fig_line.update_layout(
        title=None,
        xaxis_title="Fecha",
        yaxis_title="Precio promedio (S/)",
        hovermode="x unified",
        margin=dict(l=40, r=40, t=40, b=120),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.30,
            xanchor="center",
            x=0.5,
            bgcolor="rgba(0,0,0,0)",
        ),
    )

    fig_line.show()
    fig_line.write_image("results/Educacion/educacion_inicial/evolucion_precio_por_producto_inicial.pdf")
    
else:
    print("No hay 'fecha_scraping' válida para hacer la evolución temporal.")




# ===============================
# 7. Coeficiente de Variación (CV) por producto (gráfico que faltaba)
# ===============================
cv_prod = (
    df.groupby("producto_especifico")["precio_por_unidad_estandar"]
      .agg(["mean", "std"])
      .reset_index()
)
cv_prod["CV"] = cv_prod["std"] / cv_prod["mean"] * 100
cv_prod = cv_prod.rename(columns={"mean": "MEDIA", "std": "DESVIACION"})

print("\nCoeficiente de Variación por producto - INICIAL:")
print(cv_prod)

fig_cv = px.bar(
    cv_prod,
    x="producto_especifico",
    y="CV",
    labels={
        "producto_especifico": "Producto (INICIAL)",
        "CV": "Coeficiente de Variación (%)",
    },
    title=None,
    width=900,
    height=500,
    template="plotly_white",
)

fig_cv.update_layout(
    xaxis_title="Producto (INICIAL)",
    yaxis_title="Coeficiente de Variación (%)",
    margin=dict(l=40, r=40, t=40, b=120),
    showlegend=False,
)

fig_cv.show()
fig_cv.write_image("results/Educacion/educacion_inicial/coef_variacion_producto_inicial.pdf")


Volatilidad por producto - INICIAL (precio directo):
      producto_especifico  VOLATILIDAD
0              cartulinas     0.622846
1                 colores     0.349119
2        crayones-y-oleos     1.060373
3  cuaderno triple reglon     0.635783
4             limpiatipos     0.047502
5             plastilinas     1.158505
6                  reglas     2.900889



Coeficiente de Variación por producto - INICIAL:
      producto_especifico     MEDIA  DESVIACION          CV
0              cartulinas  1.165371    0.622846   53.446185
1                 colores  0.864312    0.349119   40.392659
2        crayones-y-oleos  1.143048    1.060373   92.767215
3  cuaderno triple reglon  5.107843    0.635783   12.447196
4             limpiatipos  2.166667    0.047502    2.192386
5             plastilinas  1.075656    1.158505  107.702245
6                  reglas  3.245390    2.900889   89.384908


## PRIMARIA

In [4]:

df = pd.read_excel("data/consolidated/consolidado_educacion.xlsx")

# Asegurar que el precio esté en formato numérico
df["precio_por_unidad_estandar"] = pd.to_numeric(
    df["precio_por_unidad_estandar"], errors="coerce"
)

# Eliminar filas sin precio válido
df = df.dropna(subset=["precio_por_unidad_estandar"]).copy()

# ===============================
# 2. Filtrar SOLO productos de PRIMARIA
# ===============================
productos_primaria = [
    "borradores",
    "colas",
    "cuaderno doble reglon",
    "cuaderno rayado",
    "forros",
    "gomas",
    "lapices",
    "tajadores",
    "tijeras",
]

df["producto_especifico"] = df["producto_especifico"].astype(str)
df = df[df["producto_especifico"].isin(productos_primaria)].copy()

# (opcional) marcar que todo es PRIMARIA
df["nivel_educativo"] = "PRIMARIA"

# ===============================
# 3. Preparar fecha
# ===============================
if "fecha_scraping" in df.columns:
    df["FECHA"] = pd.to_datetime(df["fecha_scraping"], errors="coerce")
else:
    df["FECHA"] = pd.NaT

# ===============================
# 4. Volatilidad por producto (solo PRIMARIA, precio directo)
# ===============================
vol_prod = (
    df.groupby("producto_especifico")["precio_por_unidad_estandar"]
      .std()
      .reset_index(name="VOLATILIDAD")
)

print("Volatilidad por producto - PRIMARIA (precio directo):")
print(vol_prod)

# Heatmap: una fila con N productos (estilo salud)
fig_vol = go.Figure(
    data=go.Heatmap(
        z=[vol_prod["VOLATILIDAD"].values],
        x=vol_prod["producto_especifico"],
        y=["Volatilidad"],
        colorscale="Teal",
        colorbar=dict(title="Volatilidad"),
        showscale=True,
    )
)

# Anotar valores dentro de cada celda
for _, row in vol_prod.iterrows():
    fig_vol.add_trace(
        go.Scatter(
            x=[row["producto_especifico"]],
            y=["Volatilidad"],
            text=[f"{row['VOLATILIDAD']:.2f}"],
            mode="text",
            showlegend=False,
        )
    )

fig_vol.update_layout(
    title=None,
    xaxis_title="Producto (PRIMARIA)",
    yaxis_title="",
    template="plotly_white",
    width=900,
    height=500,
    margin=dict(l=40, r=40, t=40, b=120),
)
fig_vol.show()
fig_vol.write_image("results/Educacion/educacion_primaria/volatilidad_por_producto_primaria.pdf")


# ===============================
# 5. Boxplot de precios por producto (precio directo, estilo salud)
# ===============================
fig_box = px.box(
    df,
    x="producto_especifico",
    y="precio_por_unidad_estandar",
    color="producto_especifico",
    labels={
        "producto_especifico": "Producto (PRIMARIA)",
        "precio_por_unidad_estandar": "Precio por unidad estándar (S/)",
    },
    width=900,
    height=500,
    template="plotly_white",
)

fig_box.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,  # la leyenda sería igual que el eje X
    title=None,
    margin=dict(l=40, r=40, t=40, b=120),
)

# Ordenar productos por valor total (como en el ejemplo salud)
fig_box.update_layout(xaxis={"categoryorder": "total descending"})

# Rango del eje Y (opcional tipo salud)
max_price = df["precio_por_unidad_estandar"].max()
fig_box.update_yaxes(range=[0, max_price * 1.1])

fig_box.show()
fig_box.write_image("results/Educacion/educacion_primaria/boxplot_precio_por_producto_primaria.pdf")


# ===============================
# 6. Evolución del precio promedio por producto (precio directo)
# ===============================
if df["FECHA"].notna().any():
    evol_prod = (
        df.groupby(["FECHA", "producto_especifico"])["precio_por_unidad_estandar"]
          .mean()
          .reset_index()
    )

    fig_line = px.line(
        evol_prod,
        x="FECHA",
        y="precio_por_unidad_estandar",
        color="producto_especifico",
        markers=True,
        labels={
            "FECHA": "Fecha",
            "precio_por_unidad_estandar": "Precio promedio (S/)",
            "producto_especifico": "Producto (PRIMARIA)",
        },
        width=900,
        height=500,
        template="plotly_white",
    )

    fig_line.update_layout(
        title=None,
        xaxis_title="Fecha",
        yaxis_title="Precio promedio (S/)",
        hovermode="x unified",
        margin=dict(l=40, r=40, t=40, b=120),
        legend=dict(
            orientation="h",       # horizontal
            yanchor="top",
            y=-0.30,               # leyenda debajo del gráfico
            xanchor="center",
            x=0.5,
            bgcolor="rgba(0,0,0,0)",
        ),
    )

    fig_line.show()
    fig_line.write_image("results/Educacion/educacion_primaria/evolucion_precio_por_producto_primaria.pdf")
    
else:
    print("No hay 'fecha_scraping' válida para hacer la evolución temporal.")

# ===============================
# 7. Coeficiente de Variación (CV) por producto (PRIMARIA)
# ===============================
cv_prod = (
    df.groupby("producto_especifico")["precio_por_unidad_estandar"]
      .agg(["mean", "std"])
      .reset_index()
)
cv_prod["CV"] = cv_prod["std"] / cv_prod["mean"] * 100
cv_prod = cv_prod.rename(columns={"mean": "MEDIA", "std": "DESVIACION"})

print("\nCoeficiente de Variación por producto - PRIMARIA:")
print(cv_prod)

fig_cv = px.bar(
    cv_prod,
    x="producto_especifico",
    y="CV",
    labels={
        "producto_especifico": "Producto (PRIMARIA)",
        "CV": "Coeficiente de Variación (%)",
    },
    title=None,
    width=900,
    height=500,
    template="plotly_white",
)

fig_cv.update_layout(
    xaxis_title="Producto (PRIMARIA)",
    yaxis_title="Coeficiente de Variación (%)",
    margin=dict(l=40, r=40, t=40, b=120),
    showlegend=False,
)

fig_cv.show()
fig_cv.write_image("results/Educacion/educacion_primaria/coef_variacion_producto_primaria.pdf")



Volatilidad por producto - PRIMARIA (precio directo):
     producto_especifico  VOLATILIDAD
0             borradores     1.336907
1                  colas     0.886005
2  cuaderno doble reglon     0.299377
3        cuaderno rayado     0.492877
4                 forros     2.425940
5                  gomas     0.924576
6                lapices     0.784897
7              tajadores     1.919426
8                tijeras     2.621009



Coeficiente de Variación por producto - PRIMARIA:
     producto_especifico     MEDIA  DESVIACION         CV
0             borradores  2.861158    1.336907  46.726070
1                  colas  1.668835    0.886005  53.091219
2  cuaderno doble reglon  5.227273    0.299377   5.727211
3        cuaderno rayado  5.029730    0.492877   9.799281
4                 forros  5.924648    2.425940  40.946570
5                  gomas  1.608452    0.924576  57.482334
6                lapices  1.040409    0.784897  75.441185
7              tajadores  4.295784    1.919426  44.681633
8                tijeras  4.638951    2.621009  56.500042


## SECUNDARIA

In [5]:

df = pd.read_excel("data/consolidated/consolidado_educacion.xlsx")


# Asegurar que el precio esté en formato numérico
df["precio_por_unidad_estandar"] = pd.to_numeric(
    df["precio_por_unidad_estandar"], errors="coerce"
)

# Eliminar filas sin precio válido
df = df.dropna(subset=["precio_por_unidad_estandar"]).copy()

# ===============================
# 2. Filtrar SOLO productos de SECUNDARIA
# ===============================
productos_secundaria = [
    # "calculadoras",
    "compases",
    "correctores",
    "cuaderno cuadriculado",
    "escuadras",
    "lapiceros",
    # "mochila escolar",
    "papel-bond",
    "plumones-para-papel",
    "resaltadores",
]

df["producto_especifico"] = df["producto_especifico"].astype(str)
df = df[df["producto_especifico"].isin(productos_secundaria)].copy()

# (opcional) marcar que todo es SECUNDARIA
df["nivel_educativo"] = "SECUNDARIA"

# ===============================
# 3. Preparar fecha
# ===============================
if "fecha_scraping" in df.columns:
    df["FECHA"] = pd.to_datetime(df["fecha_scraping"], errors="coerce")
else:
    df["FECHA"] = pd.NaT

# ===============================
# 4. Volatilidad por producto (SECUNDARIA, precio directo)
# ===============================
vol_prod = (
    df.groupby("producto_especifico")["precio_por_unidad_estandar"]
      .std()
      .reset_index(name="VOLATILIDAD")
)

print("Volatilidad por producto - SECUNDARIA (precio directo):")
print(vol_prod)

# Heatmap: una fila con N productos (estilo salud)
fig_vol = go.Figure(
    data=go.Heatmap(
        z=[vol_prod["VOLATILIDAD"].values],      # una fila con N productos
        x=vol_prod["producto_especifico"],       # eje X: productos
        y=["Volatilidad"],                       # eje Y: una sola fila
        colorscale="Teal",
        colorbar=dict(title="Volatilidad"),
        showscale=True,
    )
)

# Anotar valores dentro de cada celda
for _, row in vol_prod.iterrows():
    fig_vol.add_trace(
        go.Scatter(
            x=[row["producto_especifico"]],
            y=["Volatilidad"],
            text=[f"{row['VOLATILIDAD']:.2f}"],
            mode="text",
            showlegend=False,
        )
    )

fig_vol.update_layout(
    title=None,
    xaxis_title="Producto (SECUNDARIA)",
    yaxis_title="",
    template="plotly_white",
    width=900,
    height=500,
    margin=dict(l=40, r=40, t=40, b=120),
)
fig_vol.show()
fig_vol.write_image("results/Educacion/educacion_secundaria/volatilidad_por_producto_secundaria.pdf")


# ===============================
# 5. Boxplot de precios por producto (precio directo, estilo salud)
# ===============================
fig_box = px.box(
    df,
    x="producto_especifico",
    y="precio_por_unidad_estandar",
    color="producto_especifico",
    labels={
        "producto_especifico": "Producto (SECUNDARIA)",
        "precio_por_unidad_estandar": "Precio por unidad estándar (S/)",
    },
    width=900,
    height=500,
    template="plotly_white",
)

fig_box.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,  # la leyenda sería igual que el eje X
    title=None,
    margin=dict(l=40, r=40, t=40, b=120),
)

# Ordenar productos por valor total (como en el ejemplo de salud)
fig_box.update_layout(xaxis={"categoryorder": "total descending"})

# Rango del eje Y (opcional tipo salud)
max_price = df["precio_por_unidad_estandar"].max()
fig_box.update_yaxes(range=[0, max_price * 1.1])

fig_box.show()
fig_box.write_image("results/Educacion/educacion_secundaria/boxplot_precio_por_producto_secundaria.pdf")


# ===============================
# 6. Evolución del precio promedio por producto (precio directo)
# ===============================
if df["FECHA"].notna().any():
    evol_prod = (
        df.groupby(["FECHA", "producto_especifico"])["precio_por_unidad_estandar"]
          .mean()
          .reset_index()
    )

    fig_line = px.line(
        evol_prod,
        x="FECHA",
        y="precio_por_unidad_estandar",
        color="producto_especifico",
        markers=True,
        labels={
            "FECHA": "Fecha",
            "precio_por_unidad_estandar": "Precio promedio (S/)",
            "producto_especifico": "Producto (SECUNDARIA)",
        },
        width=900,
        height=500,
        template="plotly_white",
    )

    fig_line.update_layout(
        title=None,
        xaxis_title="Fecha",
        yaxis_title="Precio promedio (S/)",
        hovermode="x unified",
        margin=dict(l=40, r=40, t=40, b=120),
        legend=dict(
            orientation="h",       # horizontal
            yanchor="top",
            y=-0.30,               # leyenda debajo del gráfico
            xanchor="center",
            x=0.5,
            bgcolor="rgba(0,0,0,0)",
        ),
    )

    fig_line.show()
    fig_line.write_image(
        os.path.join(RESULTS_DIR, "evolucion_precio_por_producto_secundaria.pdf")
    )
else:
    print("No hay 'fecha_scraping' válida para hacer la evolución temporal.")

# ===============================
# 7. Coeficiente de Variación (CV) por producto (SECUNDARIA)
# ===============================
cv_prod = (
    df.groupby("producto_especifico")["precio_por_unidad_estandar"]
      .agg(["mean", "std"])
      .reset_index()
)
cv_prod["CV"] = cv_prod["std"] / cv_prod["mean"] * 100
cv_prod = cv_prod.rename(columns={"mean": "MEDIA", "std": "DESVIACION"})

print("\nCoeficiente de Variación por producto - SECUNDARIA:")
print(cv_prod)

fig_cv = px.bar(
    cv_prod,
    x="producto_especifico",
    y="CV",
    labels={
        "producto_especifico": "Producto (SECUNDARIA)",
        "CV": "Coeficiente de Variación (%)",
    },
    title=None,
    width=900,
    height=500,
    template="plotly_white",
)

fig_cv.update_layout(
    xaxis_title="Producto (SECUNDARIA)",
    yaxis_title="Coeficiente de Variación (%)",
    margin=dict(l=40, r=40, t=40, b=120),
    showlegend=False,
)

fig_cv.show()
fig_cv.write_image("results/Educacion/educacion_secundaria/coef_variacion_producto_secundaria.pdf")


Volatilidad por producto - SECUNDARIA (precio directo):
     producto_especifico  VOLATILIDAD
0               compases     2.384254
1            correctores     1.342200
2  cuaderno cuadriculado     0.671068
3              escuadras     1.349319
4              lapiceros     1.774414
5             papel-bond     0.006248
6    plumones-para-papel     0.847516
7           resaltadores     1.008130



Coeficiente de Variación por producto - SECUNDARIA:
     producto_especifico     MEDIA  DESVIACION         CV
0               compases  4.628571    2.384254  51.511667
1            correctores  2.657961    1.342200  50.497352
2  cuaderno cuadriculado  5.439655    0.671068  12.336589
3              escuadras  4.403040    1.349319  30.645169
4              lapiceros  2.434883    1.774414  72.874718
5             papel-bond  0.032225    0.006248  19.389309
6    plumones-para-papel  1.571633    0.847516  53.925850
7           resaltadores  2.643841    1.008130  38.131278
